In [8]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%pylab
import pandas as pd
%matplotlib inline

In [7]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()



ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-4811e42583e3>:4 

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
# Full original training set
raw_full_train = sc.textFile("dac/train.txt")

# Final test set split
raw_final_test = sc.textFile("dac/split/test.txt")  # Do not touch during training

# Training set splits
raw_test_3m = sc.textFile("dac/split/test_3m.txt")
raw_train_5m = sc.textFile("dac/split/train_5m.txt")
raw_validation_2m = sc.textFile("dac/split/train_5m.txt")

# Debug set
raw_small_train = sc.textFile("dac/small-train.txt")

In [5]:
# Change the data types of the datasets so that the RDD's include Int's and Strings. 

full_train = raw_full_train.map(convert_line)
final_test = raw_final_test.map(convert_line)
test_3m = raw_test_3m.map(convert_line)
train_5m = raw_train_5m.map(convert_line)
validation_2m = raw_validation_2m.map(convert_line)

debug = raw_small_train.map(convert_line)

In [4]:
#

def convert_value(index, value):
    if index < 14:
        return int(value) if value else None
    else:
        return value if value else None

def convert_line(line):
    return [convert_value(i, value) for i, value in enumerate(line.split("\t"))]

In [6]:
debug.map(lambda row: row[0]).histogram([0,1,2])
# data.map(lambda row: row[0]).histogram([0, 1, 2])
# data.map(lambda row: row[2]).histogram(10)
# a = data.map(lambda row: row[2]).histogram([-10,0,5,10,50,100,200,500,1000,2000,10000,50000])

([0, 1, 2], [77337, 22663])

In [36]:
def show_histogram(col):
    bins, counts = col.histogram(10)
    total = sum(counts)
    #Show histogram
    
    print "bins=%s, counts=%s (total=%d)" % ( bins, counts, total)
    #sum the counts
    #max of the counts
    #if  > 25%

def convertXtoString(bins):
    x = []
    for i in range(len(bins)-1):
        x.append("%s-%s" % (bins[i],bins[i+1]))
        
    print x
    return x

In [45]:
def get_column_of_rdd(col_num, data):
    return data.map(lambda row: row[col_num])

def column_integer_iter(data):
    for i in range(1,14):
         yield get_column_of_rdd(i,data)

In [37]:
for col in column_integer_iter(debug):
    show_histogram(col)

bins=[0.0, 55.6, 111.2, 166.8, 222.4, 278.0, 333.6, 389.2, 444.8, 500.40000000000003, 556], counts=[55236, 295, 34, 10, 5, 3, 2, 0, 1, 1] (total=55587)
bins=[-2.0, 1850.4, 3702.8, 5555.200000000001, 7407.6, 9260.0, 11112.400000000001, 12964.800000000001, 14817.2, 16669.600000000002, 18522], counts=[98530, 1397, 52, 12, 7, 0, 1, 0, 0, 1] (total=100000)
bins=[0.0, 6553.5, 13107.0, 19660.5, 26214.0, 32767.5, 39321.0, 45874.5, 52428.0, 58981.5, 65535], counts=[80863, 11, 1, 20, 0, 0, 0, 0, 1, 2] (total=80898)
bins=[0.0, 41.7, 83.4, 125.10000000000001, 166.8, 208.5, 250.20000000000002, 291.90000000000003, 333.6, 375.3, 417], counts=[79210, 1178, 44, 8, 12, 6, 5, 0, 0, 3] (total=80466)
bins=[0.0, 174112.8, 348225.6, 522338.39999999997, 696451.2, 870564.0, 1044676.7999999999, 1218789.5999999999, 1392902.4, 1567015.2, 1741128], counts=[93216, 1145, 491, 218, 88, 44, 19, 11, 5, 3] (total=95240)
bins=[0, 1629, 3258, 4887, 6516, 8145, 9774, 11403, 13032, 14661, 16290], counts=[74200, 536, 92, 36,

In [41]:
def provide_statistics_about_RDD(col):
    print col
    mean_val = col.mean()
    std_val = col.sampleStdev()
    #skew_val =
    return mean_val,std_val

In [49]:

for col in column_integer_iter(debug):
    print col.sampleStdev()
    break
#for col in column_iter(debug):
 #   mean, std = provide_statistics_about_RDD(col)
#   print "Mean: %s, StDev: %s" % (mean,std) 


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 161.0 failed 1 times, most recent failure: Lost task 0.0 in stage 161.0 (TID 322, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspa

In [26]:
a, b = provide_statistics_about_RDD(debug)
print "A: %s, B: %s" % (a,b) 


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 102.0 failed 1 times, most recent failure: Lost task 0.0 in stage 102.0 (TID 204, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspa